## Compare results for multiple results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    try: 
        val_dr=dict_val['hist_val'].copy()
        val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins

        sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2

        size=len(dict_val['hist_val'])
        l1,l2=int(size*0.3),int(size*0.7)
        keys=['chi_1a','chi_1b','chi_1c','chi_1']
        
        for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
            chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})

        idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
    #     chisqr_dict.update({'chi_sqr1':})

        chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
        chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data

        idx=64
        spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})

        ### computing the spectral loss chi-square
        chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
        
        spec_loss=1.0*np.log(np.mean((dict_val['spec_val'][:idx]-dict_sample['spec_val'][:idx])**2))+1.0*np.log(np.mean((dict_val['spec_err'][:idx]-dict_sample['spec_err'][:idx])**2))
        chisqr_dict.update({'chi_spec3':spec_loss})
    
    except Exception as e: 
        print(e)
        
        keys=['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
        chisqr_dict=dict.fromkeys(keys,np.nan)
        pass
    
    return chisqr_dict


In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
# s_val=np.load(fname,mmap_mode='r')[10000:20000][:,0,:,:]
s_val=np.load(fname,mmap_mode='r')[10000:18000][:,0,:,:]

print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(8000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [7]:
df_runs=pd.DataFrame([])

In [8]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:500][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [9]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['run5_fixed_cosmology','run6_fixed_cosmology','run7_no_truncated_normal','run8_no_truncated_normal']
prefix='keras_'

for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key=prefix+strg
    fname=parent_dir+fldr+'/models/gen_imgs.npy'
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:500]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [10]:
## Pytorch code
### Load images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201121_050116_full_loss_b64_lr0.0002_good_models/'
prefix='pytorch_1_'

lst=[(13, 42250), (26, 82050), (31, 98150), (36, 112600), (36, 112850), (37, 117600)]

for count,run in enumerate(lst):
    epoch,step=run[0],run[1]
    key=prefix+'{0}-{1}'.format(epoch,step)
    fname=glob.glob(parent_dir+'images/'+'*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
    
    dict1={'label':key,'fname':fname}
    images=np.load(dict1['fname'])     ### No transform needed here
    if len(images.shape)==4: ## fix for the best images being saved with an extra dimension
        print(key)
        images=images[:,0,:,:]
    print(images.shape)
    print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)



(64, 128, 128)
0.9825778 -0.9999517
(64, 128, 128)
0.98322076 -0.9997037
(64, 128, 128)
0.9824333 -0.9999228
(64, 128, 128)
0.98254544 -0.9991303
(64, 128, 128)
0.9834548 -0.99944323
(64, 128, 128)
0.9873324 -0.9996248


In [11]:
df_runs.columns

Index(['chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2', 'chi_imgvar',
       'chi_spec1', 'chi_spec2', 'chi_spec3', 'fname', 'hist_bin_centers',
       'hist_err', 'hist_val', 'label', 'num_img', 'spec_err', 'spec_val'],
      dtype='object')

In [17]:

col_list=['label', 'chi_1', 'chi_1a', 'chi_spec1', 'chi_spec3','chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','num_img','fname']

df_runs=df_runs.reindex(columns=col_list)


In [18]:
df_runs

,label,chi_1,chi_1a,chi_spec1,chi_spec3,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,num_img,fname
0,raw,0.014575,0.000112,0.001910,19.401889,0.000187,0.014276,0.000020,4.080889,0.112023,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0028445197484184732, 0.0020816260811514454,...","[1.121347563774932, 0.9576931243390049, 0.2568...","[44450.16800148834, 22573.110844265062, 15349....","[87592.04888481903, 62024.13159096335, 44898.4...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.001702,0.000073,0.052795,23.822982,0.000449,0.001180,0.000055,4.072555,5.345283,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.003081765924486072, 0.0023479649643570986, ...","[1.1146491994998429, 0.9643402305198981, 0.260...","[42790.708161353, 21535.157497092187, 14415.30...","[85906.63101111341, 60940.734434269594, 43971....",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.002058,0.000455,0.066858,21.185615,0.000231,0.001373,0.000113,4.171513,5.868547,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0031456442853499706, 0.002349065715521853, ...","[1.1148820753244182, 0.9669777732485703, 0.261...","[44993.59687564774, 22703.883970669194, 16263....","[86439.3065150423, 60426.5470668808, 45079.825...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
3,keras_2,0.003165,0.002139,0.020309,25.792137,0.000352,0.000674,0.001705,4.130929,1.242998,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0032299157726108983, 0.00241667307354839, 0...","[1.1439634352627102, 0.9270007224057393, 0.253...","[41984.63453742358, 22377.328750660476, 14242....","[82519.1559628191, 59058.62437103868, 43854.19...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
4,keras_3,0.003478,0.001618,0.047763,20.922330,0.000507,0.001353,0.001206,4.001797,5.103600,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0029680794378720322, 0.002208163391797882, ...","[1.0973298711470636, 0.9833819747814072, 0.269...","[45458.16712303361, 21884.70908105985, 14530.5...","[88968.60505310107, 62091.84885758681, 45370.3...",500.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
5,pytorch_1_13-42250,0.092926,0.066867,3.521283,31.390942,0.025182,0.000877,0.019457,12.624207,71.236188,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.007463251075644719, 0.005014208342821668, 0...","[1.1453971296290497, 0.8407663119869533, 0.295...","[69689.62432503577, 30523.771846611344, 24093....","[92171.87540227175, 60912.84650745765, 51456.6...",64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
6,pytorch_1_26-82050,0.009799,0.002257,0.250944,26.832139,0.005235,0.002307,0.001891,8.597163,25.445298,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.005591630501843404, 0.006078790571152159, 0...","[1.138934615211898, 0.9216316952597118, 0.2640...","[40845.722680483384, 20937.718097760604, 14267...","[81596.33827212453, 63937.60810701549, 47467.6...",64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
7,pytorch_1_31-98150,0.008963,0.004511,0.763103,28.874139,0.002254,0.002197,0.002996,9.149273,83.370757,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.00603209477730977, 0.0063741235330733665, 0...","[1.1445409685092875, 0.9118746235112072, 0.266...","[40031.213394994935, 22804.046851621944, 12270...","[73955.27607680485, 55909.67867641897, 41194.8...",64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
8,pytorch_1_36-112600,0.009476,0.005421,1.379442,28.785039,0.001892,0.002162,0.004623,9.606555,85.585731,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.006253453109560236, 0.004213783558638739, 0...","[1.1550439625968942, 0.9022355689595399, 0.264...","[41279.193166640696, 21784.353581711464, 12495...","[91979.61307770014, 52228.39206728338, 34690.1...",64.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
9,pytorch_1_36-112850,0.126804,0.113891,1.735579,31.399476,0.012151,0.000763,0.044717,15.313786,11.679441,"[-1.031746031584782, -0.6161616

### Compare chi-squares

In [19]:
chi_keys=['label','chi_1','chi_spec1', 'num_img', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','chi_spec3']
df_runs[chi_keys]

,label,chi_1,chi_spec1,num_img,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,chi_spec3
0,raw,0.014575,0.001910,500.0,0.000112,0.000187,0.014276,0.000020,4.080889,0.112023,19.401889
1,keras_0,0.001702,0.052795,500.0,0.000073,0.000449,0.001180,0.000055,4.072555,5.345283,23.822982
2,keras_1,0.002058,0.066858,500.0,0.000455,0.000231,0.001373,0.000113,4.171513,5.868547,21.185615
3,keras_2,0.003165,0.020309,500.0,0.002139,0.000352,0.000674,0.001705,4.130929,1.242998,25.792137
4,keras_3,0.003478,0.047763,500.0,0.001618,0.000507,0.001353,0.001206,4.001797,5.103600,20.922330
5,pytorch_1_13-42250,0.092926,3.521283,64.0,0.066867,0.025182,0.000877,0.019457,12.624207,71.236188,31.390942
6,pytorch_1_26-82050,0.009799,0.250944,64.0,0.002257,0.005235,0.002307,0.001891,8.597163,25.445298,26.832139
7,pytorch_1_31-98150,0.008963,0.763103,64.0,0.004511,0.002254,0.002197,0.002996,9.149273,83.370757,28.874139
8,pytorch_1_36-112600,0.009476,1.379442,64.0,0.005421,0.001892,0.002162,0.004623,9.606555,85.585731,28.785039
9,pytorch_1_36-112850,0.126804,1.735579,64.0,0.113891,0.012151,0.000763,0.044717,15.313786,11.679441,31.399476


## Plot

In [20]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    
#     plt.xscale('symlog',linthreshx=30)

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [21]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_runs),
                labels_list=SelectMultiple(options=df_runs.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

interactive(children=(SelectMultiple(description='labels_list', options=('raw', 'keras_0', 'keras_1', 'keras_2…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [32]:
col_list=['label','chi_1','chi_spec1','num_img','chi_2','chi_spec2']
df_runs[col_list].head(10)

,label,chi_1,chi_spec1,num_img,chi_2,chi_spec2
0,raw,0.014575,0.001910,500.0,0.000020,56.011413
1,keras_0,0.001702,0.052795,500.0,0.000055,2672.641410
2,keras_1,0.002058,0.066858,500.0,0.000113,2934.273632
3,keras_2,0.003165,0.020309,500.0,0.001705,621.498995
4,keras_3,0.003478,0.047763,500.0,0.001206,2551.799936
5,pytorch_1_2-6662,0.565152,15.742224,2000.0,0.294867,580068.234549
6,pytorch_1_3-11089,0.151757,3.611361,2000.0,0.045627,308707.589729
7,pytorch_1_11-34800,0.029449,0.718692,64.0,0.010006,1229.065931
8,pytorch_1_11-35200,0.027141,0.486636,64.0,0.003830,1850.241459
9,pytorch_1_12-37800,0.031397,0.743896,64.0,0.005195,3849.303141
